- 保监会 相关性模型 2 训练 

# 基本设置

In [1]:
##load packages, needed
# encoding=utf-8

import jieba
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif 

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import joblib
%matplotlib inline
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
class StatsFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length])            
        return data

## 上一版模型读入

In [3]:
# 上一版模型
# from sklearn.externals import joblib
# pipeline_old = joblib.load( "model/circ_cor_0625.pkl.z") 

# 导入数据

In [4]:
title = []
filename = 'data/titles.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    title.append(f.strip().replace('\n', ''))
fid.close()
print(len(title))

5807


In [5]:
content = []
filename = 'data/contents.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    content.append(f.strip().replace('\n', ''))
fid.close()
print(len(content))
# content[:2]

5807


In [6]:
title_content = [t + ' ' + c for t,c in zip(title, content)]
print(len(title_content))

5807


In [7]:
label = []
filename = 'data/labels.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    label.append(f.strip().replace('\n', ''))
fid.close()
print(len(label))
label[:5]

5807


['1', '1', '1', '1', '1']

# 训练模型

## 分割训练集、测试集

In [8]:
X_train, X_test, y_train, y_test = train_test_split(title_content, label, test_size=0.3, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 4064
训练集-各类数量： Counter({'0': 2067, '1': 1997})
测试集： 1743
测试集-各类数量： Counter({'1': 883, '0': 860})


## xgboost

### train

In [9]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer()),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('len_stats', StatsFeatures())
    ])),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=2))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9768700787401575


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

### test

In [10]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.861732644865175
             precision    recall  f1-score   support

          0       0.84      0.89      0.86       860
          1       0.88      0.84      0.86       883

avg / total       0.86      0.86      0.86      1743

confusion_matrix: 
[[764  96]
 [145 738]]


In [11]:
# 上一版模型 
# y_pred_class = pipeline_old.predict(X_test)
# print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
# print(metrics.classification_report(y_test, y_pred_class))
# print('confusion_matrix: ')
# print( metrics.confusion_matrix(y_test, y_pred_class))

### KFold

In [14]:
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler

kf = KFold(n_splits=5, shuffle=True, random_state=0)
title_content = np.array(title_content)
label = np.array(label)

In [15]:
num = 0
for train_index , test_index in kf.split(title_content):
    print('---- %s'%(num+1))
    X_train,X_test = title_content[train_index], title_content[test_index]
    y_train,y_test = label[train_index], label[test_index]
    
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('tf_idf', Pipeline([
                ('counts', CountVectorizer(max_df=0.95, min_df=2)),
                ('tf_idf', TfidfTransformer()),
                ('chi', SelectKBest(chi2, k=20000))
            ])),
            ('len_stats', StatsFeatures())
        ])),
        ('standard', StandardScaler(with_mean=False)),
        ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
    ])
    pipeline.fit(X_train, y_train)
    print(pipeline.score(X_train, y_train))    
    
    y_pred_class = pipeline.predict(X_test)
    print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
    print(metrics.classification_report(y_test, y_pred_class))
    print('confusion_matrix: ')
    print( metrics.confusion_matrix(y_test, y_pred_class))    
    
    num += 1

---- 1
0.9741657696447793
accuracy_score:  0.8898450946643718
             precision    recall  f1-score   support

          0       0.89      0.90      0.89       597
          1       0.89      0.88      0.89       565

avg / total       0.89      0.89      0.89      1162

confusion_matrix: 
[[537  60]
 [ 68 497]]
---- 2
0.976318622174381
accuracy_score:  0.8623063683304647
             precision    recall  f1-score   support

          0       0.85      0.88      0.86       575
          1       0.88      0.84      0.86       587

avg / total       0.86      0.86      0.86      1162

confusion_matrix: 
[[508  67]
 [ 93 494]]
---- 3
0.9754627636676712
accuracy_score:  0.8570198105081827
             precision    recall  f1-score   support

          0       0.84      0.88      0.86       564
          1       0.88      0.84      0.86       597

avg / total       0.86      0.86      0.86      1161

confusion_matrix: 
[[495  69]
 [ 97 500]]
---- 4
0.9776151528196297
accuracy_score:  0

# 模型保存

In [12]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/mfa_sen_0830.pkl.z")  

['model/mfa_sen_0830.pkl.z']

# 保存本文件

In [56]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)